In [2]:
#!pip install ultralytics

In [2]:
import ultralytics
ultralytics.__version__

'8.1.20'

In [3]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov8s.pt')

In [4]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 
              'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [5]:
tracker=Tracker()
count=0

In [6]:
import os
import time
cap=cv2.VideoCapture('vehicles.mp4')
down = {}
up = {}
counter_down = []
counter_up = []
vehicle_elapsed_time = {}
vehicle_elapsed_time_up = {}

red_line_y = 225
blue_line_y = 325
offset = 6

# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    # if count % 2 != 0:
    #     continue
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
      #   if 'car' in c:
        list.append([x1, y1, x2, y2])
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2
        area_1 = np.array([[(920, 325) , (100, 325),(0,415),(1020,415)]], dtype=np.int32)
        area_2 = np.array([[ (720,225),(260, 225),(350,180),(600,180)]], dtype=np.int32)

        if red_line_y<(cy+offset) and red_line_y > (cy-offset):
           down[id]=time.time()   # current time when vehichle touch the first line
        if id in down:
           result = cv2.pointPolygonTest(np.array(area_1,np.int32),(cx,cy),False)
           if result>=0:
            # if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
               elapsed_time=time.time() - down[id]  # current time when vehicle touch the second line. Also we a re minusing the previous time ( current time of line 1)
               if id not in vehicle_elapsed_time:
                   vehicle_elapsed_time[id] = elapsed_time
               if id in vehicle_elapsed_time:
                   elapsed_time= vehicle_elapsed_time[id]
               distance = 10 # meters 
               a_speed_ms = distance / elapsed_time
               a_speed_kh = a_speed_ms * 3.6  # this will give kilometers per hour for each vehicle. This is the condition for going downside
               print("down speed is ")
               print(a_speed_kh)
               if counter_down.count(id)==0:
                  counter_down.append(id)
               cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
               cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
               cv2.putText(frame,"Vehicle "+str(len(counter_down)),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
               cv2.putText(frame,"%.2f" % a_speed_kh + 'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
             

                
        #####going UP blue line#####     
        if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
           up[id]=time.time()
        if id in up:
           result2 = cv2.pointPolygonTest(np.array(area_2,np.int32),(cx,cy),False)
         #   if red_line_y<(cy+offset) and red_line_y > (cy-offset):
           if result2>=0:
               elapsed1_time=time.time() - up[id]
               if id not in vehicle_elapsed_time_up:
                   vehicle_elapsed_time_up[id] = elapsed1_time
               if id in vehicle_elapsed_time_up:
                   elapsed1_time= vehicle_elapsed_time_up[id]
               
               print("elapsed_time is ")
               print(elapsed1_time)
               distance1 = 10 # meters  (Distance between the 2 lines is 10 meters )
               a_speed_ms1 = distance1 / elapsed1_time
               a_speed_kh1 = a_speed_ms1 * 3.6
               print("upgoing speed is ")
               print(a_speed_kh1)
               # formula of speed= distance/time 
               if counter_up.count(id)==0:
                  counter_up.append(id)      
                  
               cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
               cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
               cv2.putText(frame,"Vehicle "+str(len(counter_up)),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
               cv2.putText(frame,"%.2f" % a_speed_kh1 + 'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



    
    text_color = (0, 0, 0)  # Black color for text
    yellow_color = (0, 255, 0)  # Yellow color for background
    red_color = (0, 0, 255)  # Red color for lines
    blue_color = (0, 255, 0)  # Blue color for lines

    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)
    cv2.polylines(frame, [area_1], isClosed=True, color=(0,255,0), thickness=2)
    cv2.line(frame, (260, 225), (720,225), red_color,1)
   #  cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.polylines(frame, [area_2], isClosed=True, color=(0,0,255), thickness=2)

    cv2.line(frame, (135, 325), (920, 325), blue_color, 1)
    # cv2.putText(frame, ('Blue Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Save frame
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)

    out.write(frame)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
    #if cv2.waitKey(0) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 320x640 4 cars, 1 truck, 108.3ms
Speed: 5.5ms preprocess, 108.3ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 1 truck, 104.7ms
Speed: 0.5ms preprocess, 104.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 1 bus, 1 truck, 94.9ms
Speed: 5.3ms preprocess, 94.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 1 truck, 94.2ms
Speed: 2.7ms preprocess, 94.2ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 1 truck, 94.6ms
Speed: 1.1ms preprocess, 94.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 1 truck, 90.8ms
Speed: 4.7ms preprocess, 90.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 1 bus, 1 truck, 91.1ms
Speed: 1.1ms preprocess, 91.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 truck, 93.2ms
Speed: 15.6ms p